In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!python --version

Python 3.10.12


In [ ]:
!nvidia-smi

Fri Dec 22 01:31:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10                     On  | 00000000:07:00.0 Off |                    0 |
|  0%   30C    P8              15W / 150W |      4MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           196Gi       1.0Gi       194Gi       5.0Mi       1.2Gi       194Gi
Swap:             0B          0B          0B


In [ ]:
 !python -c 'import torch ; print("Is available: ", torch.cuda.is_available()) ; print("Current Device: ", torch.cuda.current_device()) ; print("Pytorch CUDA Compiled version: ", torch._C._cuda_getCompiledVersion()) ; print("Pytorch version: ", torch.version) ; print("pytorch file: ", torch.__file__)'

Is available:  True
Current Device:  0
Pytorch CUDA Compiled version:  12010
Pytorch version:  <module 'torch.version' from '/home/ubuntu/.local/lib/python3.10/site-packages/torch/version.py'>
pytorch file:  /home/ubuntu/.local/lib/python3.10/site-packages/torch/__init__.py


In [ ]:
!pip install fastprogress

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
!pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 121.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 125.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 136.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 21.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 55.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 79.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 55.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 41.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 38.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import csv
import torch
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

import pandas as pd

import torch.nn as nn
import torch.optim as optim

import numpy as np
import h5py

from fastprogress import master_bar, progress_bar

In [ ]:
# Custom char tokenizer
def char_tokenizer(text):
    return list(text)

# Yield tokens from dataset for vocabulary building
def yield_tokens(data):
    for src, trg in data:
        yield char_tokenizer(src)
        yield char_tokenizer(trg)

# Load data from CSV
def load_data_from_csv(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        data = [(row[0], row[1]) for row in csv_reader]
    return data

In [ ]:
train_data_raw = load_data_from_csv('train.csv')
val_data_raw = load_data_from_csv('val.csv')
test_data_raw = load_data_from_csv('test.csv')

In [ ]:
vocab = build_vocab_from_iterator(yield_tokens(train_data_raw), specials=['<sos>', '<eos>', '<unk>'])
vocab.set_default_index(vocab["<unk>"])

print(f"Vocab length: {len(vocab)}")

Vocab length: 4131


In [ ]:
# save vocab
import pickle

with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [ ]:
# Dataset class
class CSVDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        src, trg = self.data[index]
        src = [self.vocab[token] for token in char_tokenizer(src)]
        trg = [self.vocab['<sos>']] + [self.vocab[token] for token in char_tokenizer(trg)]  # prepend <sos>
        return torch.tensor(src), torch.tensor(trg)

In [ ]:
# Padding function
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, padding_value=vocab['<eos>'], batch_first=True)
    trg_batch = pad_sequence(trg_batch, padding_value=vocab['<eos>'], batch_first=True)
    return src_batch, trg_batch

In [ ]:
# DataLoader
BATCH_SIZE = 512

train_dataset = CSVDataset(train_data_raw, vocab)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)

val_dataset = CSVDataset(val_data_raw, vocab)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)  # Usually, no need to shuffle val/test data

test_dataset = CSVDataset(test_data_raw, vocab)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)

In [ ]:
# check sample from dataset
for src, trg in train_dataset:
    print(src, trg)
    break

tensor([ 3,  7, 14, 16,  7,  4, 15, 21, 21, 12,  7, 16, 23,  6, 10, 11,  5, 14]) tensor([ 0, 14,  5, 41, 16,  6,  5,  7,  4, 15])


In [ ]:
# Sample usage:
for src, trg in train_loader:
    # src and trg here are tokenized, batched, and padded sequences
    print(src.shape, trg.shape)
    break

torch.Size([512, 42]) torch.Size([512, 31])


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len, batch_size = trg.shape
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[0, :]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = torch.rand(1) < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from IPython.utils import io

# Assuming src and trg are your tokenized, padded source and target sequences
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)

lr = 0.001

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60, eta_min=1e-5)
criterion = nn.CrossEntropyLoss()
early_stopper = EarlyStopper(patience=10)

epochs=5

mb = master_bar(range(epochs))
mb.names = ['Training loss', 'Validation loss']

x = []
training_losses = []
validation_losses = []

valid_mean_min = np.Inf

# Training loop
for epoch in mb:
  x.append(epoch)
  with io.capture_output() as captured:
      %store x
  total_loss = torch.Tensor([0.0]).to(device)

  # train
  total_train_batches = len(train_loader)
  with io.capture_output() as captured:
      %store total_train_batches
  tbc = 0
  model.train()
  for batch in progress_bar(train_loader, parent=mb):
      tbc = tbc + 1
      with io.capture_output() as captured:
          %store tbc
      src, trg = batch
      src = src.permute(1, 0).to(device).long()
      trg = trg.permute(1, 0).to(device).long()
      optimizer.zero_grad()
      output = model(src, trg)
      output_dim = output.shape[-1]
      output = output[1:].reshape(-1, output_dim)
      trg = trg[1:].reshape(-1)
      loss = criterion(output, trg)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()

  # decay lr
  scheduler.step()
  mean = total_loss / len(train_loader)
  training_losses.append(mean.cpu())
  with io.capture_output() as captured:
      %store training_losses

  # validation
  model.eval()
  total_val_batches = len(val_loader)
  with io.capture_output() as captured:
      %store total_val_batches
  validation_loss = torch.Tensor([0.0]).to(device)
  vbc = 0
  with torch.no_grad():
    for batch in progress_bar(val_loader, parent=mb):
      vbc = vbc + 1
      with io.capture_output() as captured:
          %store vbc
      src, trg = batch
      src = src.permute(1, 0).to(device).long()
      trg = trg.permute(1, 0).to(device).long()
      output = model(src, trg)
      output_dim = output.shape[-1]
      output = output[1:].reshape(-1, output_dim)
      trg = trg[1:].reshape(-1)
      loss = criterion(output, trg)
      validation_loss += loss.item()

  val_mean = validation_loss / len(val_loader)
  validation_losses.append(val_mean.cpu())
  with io.capture_output() as captured:
      %store validation_losses
  mb.update_graph([[x, training_losses], [x, validation_losses]], [0,epochs])
  mb.write(f"\nEpoch {epoch}: Training loss {mean.item():.6f} validation loss {val_mean.item():.6f} with lr {lr:.6f}")

  # save model if validation loss has decreased
  if val_mean.item() <= valid_mean_min:
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
      valid_mean_min,
      val_mean.item()))
      torch.save(model.state_dict(), 'pass_predict.pt')
      valid_mean_min = val_mean.item()

  if early_stopper.early_stop(val_mean.item()):
    break

In [ ]:
print("done")

done


In [ ]:
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
model.load_state_dict(torch.load('pass_predict.pt'))

<All keys matched successfully>

In [ ]:
itos = vocab.get_itos()

In [ ]:
itos[0]

'<sos>'

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

model.eval()
total_bleu = 0

with torch.no_grad():
    for src, trg in test_loader:
        src = src.permute(1, 0).to(device).long()
        trg = trg.permute(1, 0).to(device).long()

        # Turn off tea
        cher forcing
        output = model(src, trg, 0)
        output_dim = output.shape[-1]
        output = output[1:].reshape(-1, output_dim)

        # Convert model's outputs to tokens
        predicted_tokens = torch.argmax(output, dim=1)
        predicted_sentence = [itos[tok.item()] for tok in predicted_tokens]

        reference = [itos[tok.item()] for tok in trg[1:].reshape(-1)]
        total_bleu += sentence_bleu([reference], predicted_sentence)

average_bleu = total_bleu / len(test_loader)
print(f'Average BLEU score: {average_bleu:.3f}')

Average BLEU score: 0.719


In [ ]:
def predict(model, vocab, device, src_sentence, n=1):
    model.eval()

    # Convert the source sentence to tokens
    src_tokens = [tok for tok in src_sentence]
    src_indexes = [vocab[tok] for tok in src_tokens] + [vocab['<eos>']]

    # Convert to tensor and add batch dimension
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # Initialize target sequence with <sos> token
    result = []
    for i in range(n):
        trg_indexes = [vocab['<sos>']]
        predicted_sentence = []
        # Predict next token
        while True:
            trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1).to(device)
            with torch.no_grad():
                output = model(src_tensor, trg_tensor)
                #print(output)
                predicted_token = output.argmax(2)[-1, :].item()
                # Check for end of sentence
                # Append the predicted token to the target sequence
                trg_indexes.append(predicted_token)
                if predicted_token == vocab['<eos>']:
                    break

                # Append the predicted token to the target sequence
                trg_indexes.append(predicted_token)
                # Convert the target sequence to words
                predicted_sentence.append(itos[predicted_token])
        if ''.join(predicted_sentence[1:]) in result:
            continue
        result.append(''.join(predicted_sentence[1:]))

    return result  # Exclude the <sos> token

In [ ]:
predict(model, vocab, device, "pyro.archvile@googlemail.com", 100)

['ean',
 'ara',
 'eera',
 'erae',
 'aer',
 'ann',
 'era',
 'err',
 'aan',
 'aee',
 'anr',
 'aar',
 'een',
 'ear',
 'ana',
 'ere',
 'eer',
 'er',
 'ari',
 'erern',
 'eae',
 'erne',
 'aen',
 'ane',
 'ern',
 'ea',
 'eria',
 'ar',
 'eeaa',
 'aara',
 'aaa',
 'erin',
 'are',
 'eaa',
 'arae',
 'aa1',
 'aae',
 'aena',
 'erre',
 'erer',
 'eri']

In [ ]:
!head -10 test.csv

arturo100@rubi.net,tequiero1
mahei001@mail.ru,йцувыфячс
machomiscellanykml4@yandex.ru,iEW843lGo
angieg@tm.net.my,c_sunstrom
js152008@yahoo.com,boots5
jenia.kuznetzova@hotmail.ru,jenia12345jenia
pucoy_gunawan@yahoo.com,Janganbuka123
naidu35pharma@gmail.com,malleswari
tailincoln@cox.net,misrules26
jayeshjagetiya83@gmail.com,969858


In [ ]:
model.eval()
src_sentence = "grae123@yahoo.com"
src_tokens = [tok for tok in src_sentence]
src_indexes = [vocab[tok] for tok in src_tokens]
print(src_indexes)
src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
trg_indexes = [vocab['<sos>']]
trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1).to(device)
with torch.no_grad():
  output = model(src_tensor, trg_tensor)
print(output)
output_dim = output.shape[-1]
output = output[1:].reshape(-1, output_dim)

[26, 7, 3, 5, 15, 19, 27, 12, 20, 3, 18, 4, 4, 10, 13, 4, 8]
tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')


In [ ]:
predicted_tokens = torch.argmax(output, dim=1)

predicted_sentence = [itos[tok.item()] for tok in predicted_tokens]
predicted_sentence

[]

In [ ]:
with torch.no_grad():
    for src, trg in test_loader:
        src = src.permute(1, 0).to(device).long()
        trg = trg.permute(1, 0).to(device).long()

        # Turn off teacher forcing
        output = model(src, trg, 0)
        output_dim = output.shape[-1]
        output = output[1:].reshape(-1, output_dim)

        # Convert model's outputs to tokens
        predicted_tokens = torch.argmax(output, dim=1)
        predicted_sentence = ''.join([itos[tok.item()] for tok in predicted_tokens])

        reference = ''.join([itos[tok.item()] for tok in trg[1:].reshape(-1)])
        print(predicted_sentence)
        print(reference)
        break

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!python --version

Python 3.10.12


In [ ]:
!nvidia-smi

Fri Dec 22 01:31:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10                     On  | 00000000:07:00.0 Off |                    0 |
|  0%   30C    P8              15W / 150W |      4MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           196Gi       1.0Gi       194Gi       5.0Mi       1.2Gi       194Gi
Swap:             0B          0B          0B


In [ ]:
 !python -c 'import torch ; print("Is available: ", torch.cuda.is_available()) ; print("Current Device: ", torch.cuda.current_device()) ; print("Pytorch CUDA Compiled version: ", torch._C._cuda_getCompiledVersion()) ; print("Pytorch version: ", torch.version) ; print("pytorch file: ", torch.__file__)'

Is available:  True
Current Device:  0
Pytorch CUDA Compiled version:  12010
Pytorch version:  <module 'torch.version' from '/home/ubuntu/.local/lib/python3.10/site-packages/torch/version.py'>
pytorch file:  /home/ubuntu/.local/lib/python3.10/site-packages/torch/__init__.py


In [ ]:
!pip install fastprogress

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
!pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 121.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 125.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 136.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 21.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 55.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 79.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 55.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 41.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 38.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import csv
import torch
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

import pandas as pd

import torch.nn as nn
import torch.optim as optim

import numpy as np
import h5py

from fastprogress import master_bar, progress_bar

In [ ]:
# Custom char tokenizer
def char_tokenizer(text):
    return list(text)

# Yield tokens from dataset for vocabulary building
def yield_tokens(data):
    for src, trg in data:
        yield char_tokenizer(src)
        yield char_tokenizer(trg)

# Load data from CSV
def load_data_from_csv(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        data = [(row[0], row[1]) for row in csv_reader]
    return data

In [ ]:
train_data_raw = load_data_from_csv('train.csv')
val_data_raw = load_data_from_csv('val.csv')
test_data_raw = load_data_from_csv('test.csv')

In [ ]:
vocab = build_vocab_from_iterator(yield_tokens(train_data_raw), specials=['<sos>', '<eos>', '<unk>'])
vocab.set_default_index(vocab["<unk>"])

print(f"Vocab length: {len(vocab)}")

Vocab length: 4131


In [ ]:
# save vocab
import pickle

with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [ ]:
# Dataset class
class CSVDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        src, trg = self.data[index]
        src = [self.vocab[token] for token in char_tokenizer(src)]
        trg = [self.vocab['<sos>']] + [self.vocab[token] for token in char_tokenizer(trg)]  # prepend <sos>
        return torch.tensor(src), torch.tensor(trg)

In [ ]:
# Padding function
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, padding_value=vocab['<eos>'], batch_first=True)
    trg_batch = pad_sequence(trg_batch, padding_value=vocab['<eos>'], batch_first=True)
    return src_batch, trg_batch

In [ ]:
# DataLoader
BATCH_SIZE = 512

train_dataset = CSVDataset(train_data_raw, vocab)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)

val_dataset = CSVDataset(val_data_raw, vocab)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)  # Usually, no need to shuffle val/test data

test_dataset = CSVDataset(test_data_raw, vocab)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)

In [ ]:
# check sample from dataset
for src, trg in train_dataset:
    print(src, trg)
    break

tensor([ 3,  7, 14, 16,  7,  4, 15, 21, 21, 12,  7, 16, 23,  6, 10, 11,  5, 14]) tensor([ 0, 14,  5, 41, 16,  6,  5,  7,  4, 15])


In [ ]:
# Sample usage:
for src, trg in train_loader:
    # src and trg here are tokenized, batched, and padded sequences
    print(src.shape, trg.shape)
    break

torch.Size([512, 42]) torch.Size([512, 31])


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len, batch_size = trg.shape
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[0, :]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = torch.rand(1) < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from IPython.utils import io

# Assuming src and trg are your tokenized, padded source and target sequences
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)

lr = 0.001

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60, eta_min=1e-5)
criterion = nn.CrossEntropyLoss()
early_stopper = EarlyStopper(patience=10)

epochs=5

mb = master_bar(range(epochs))
mb.names = ['Training loss', 'Validation loss']

x = []
training_losses = []
validation_losses = []

valid_mean_min = np.Inf

# Training loop
for epoch in mb:
  x.append(epoch)
  with io.capture_output() as captured:
      %store x
  total_loss = torch.Tensor([0.0]).to(device)

  # train
  total_train_batches = len(train_loader)
  with io.capture_output() as captured:
      %store total_train_batches
  tbc = 0
  model.train()
  for batch in progress_bar(train_loader, parent=mb):
      tbc = tbc + 1
      with io.capture_output() as captured:
          %store tbc
      src, trg = batch
      src = src.permute(1, 0).to(device).long()
      trg = trg.permute(1, 0).to(device).long()
      optimizer.zero_grad()
      output = model(src, trg)
      output_dim = output.shape[-1]
      output = output[1:].reshape(-1, output_dim)
      trg = trg[1:].reshape(-1)
      loss = criterion(output, trg)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()

  # decay lr
  scheduler.step()
  mean = total_loss / len(train_loader)
  training_losses.append(mean.cpu())
  with io.capture_output() as captured:
      %store training_losses

  # validation
  model.eval()
  total_val_batches = len(val_loader)
  with io.capture_output() as captured:
      %store total_val_batches
  validation_loss = torch.Tensor([0.0]).to(device)
  vbc = 0
  with torch.no_grad():
    for batch in progress_bar(val_loader, parent=mb):
      vbc = vbc + 1
      with io.capture_output() as captured:
          %store vbc
      src, trg = batch
      src = src.permute(1, 0).to(device).long()
      trg = trg.permute(1, 0).to(device).long()
      output = model(src, trg)
      output_dim = output.shape[-1]
      output = output[1:].reshape(-1, output_dim)
      trg = trg[1:].reshape(-1)
      loss = criterion(output, trg)
      validation_loss += loss.item()

  val_mean = validation_loss / len(val_loader)
  validation_losses.append(val_mean.cpu())
  with io.capture_output() as captured:
      %store validation_losses
  mb.update_graph([[x, training_losses], [x, validation_losses]], [0,epochs])
  mb.write(f"\nEpoch {epoch}: Training loss {mean.item():.6f} validation loss {val_mean.item():.6f} with lr {lr:.6f}")

  # save model if validation loss has decreased
  if val_mean.item() <= valid_mean_min:
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
      valid_mean_min,
      val_mean.item()))
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab/password_predict/pass_predict.pt')
      valid_mean_min = val_mean.item()

  if early_stopper.early_stop(val_mean.item()):
    break

In [ ]:
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab/password_predict/pass_predict.pt'))

<All keys matched successfully>

In [ ]:
itos = vocab.get_itos()

In [ ]:
itos[0]

'<sos>'

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

model.eval()
total_bleu = 0

with torch.no_grad():
    for src, trg in test_loader:
        src = src.permute(1, 0).to(device).long()
        trg = trg.permute(1, 0).to(device).long()

        # Turn off teacher forcing
        output = model(src, trg, 0)
        output_dim = output.shape[-1]
        output = output[1:].reshape(-1, output_dim)

        # Convert model's outputs to tokens
        predicted_tokens = torch.argmax(output, dim=1)
        predicted_sentence = [itos[tok.item()] for tok in predicted_tokens]

        reference = [itos[tok.item()] for tok in trg[1:].reshape(-1)]
        total_bleu += sentence_bleu([reference], predicted_sentence)

average_bleu = total_bleu / len(test_loader)
print(f'Average BLEU score: {average_bleu:.3f}')

Average BLEU score: 0.719


In [ ]:
def predict(model, vocab, device, src_sentence, n=1):
    model.eval()

    # Convert the source sentence to tokens
    src_tokens = [tok for tok in src_sentence]
    src_indexes = [vocab[tok] for tok in src_tokens] + [vocab['<eos>']]

    # Convert to tensor and add batch dimension
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # Initialize target sequence with <sos> token
    result = []
    for i in range(n):
        trg_indexes = [vocab['<sos>']]
        predicted_sentence = []
        # Predict next token
        while True:
            trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1).to(device)
            with torch.no_grad():
                output = model(src_tensor, trg_tensor)
                #print(output)
                predicted_token = output.argmax(2)[-1, :].item()
                # Check for end of sentence
                # Append the predicted token to the target sequence
                trg_indexes.append(predicted_token)
                if predicted_token == vocab['<eos>']:
                    break

                # Append the predicted token to the target sequence
                trg_indexes.append(predicted_token)
                # Convert the target sequence to words
                predicted_sentence.append(itos[predicted_token])
        if ''.join(predicted_sentence[1:]) in result:
            continue
        result.append(''.join(predicted_sentence[1:]))

    return result  # Exclude the <sos> token

In [ ]:
predict(model, vocab, device, "pyro.archvile@googlemail.com", 100)

['aere',
 'aels',
 'ailer',
 'aell',
 'ana',
 'aie',
 'aea',
 'ail',
 'anc',
 'aia',
 'aee',
 'ain',
 'aeon',
 'aio',
 'ane',
 'aear',
 'ano',
 'aier',
 'aeer',
 'anar',
 'ance',
 'aic',
 'aer',
 'aera',
 'ael',
 'anan']

In [ ]:
!head -10 /content/drive/MyDrive/Colab/password_predict/data/test.csv

user,pass
bikramojha@yandex.ru,bhp455614
adjoa.mansa@yahoo.com,rbsfdplc
pyro.archvile@googlemail.com,Omega666
red_head76_2000@yahoo.com,0002_67daeh_der
laurentnancy@hotmail.fr,27091978zxc
19ol64@mail.ru,785333
grae123@yahoo.com,enzofan1
gulnora_nabiyeva@yahoo.com,1qaz2wsx3edc
wydik@ya.ru,525252


In [ ]:
model.eval()
src_sentence = "grae123@yahoo.com"
src_tokens = [tok for tok in src_sentence]
src_indexes = [vocab[tok] for tok in src_tokens]
print(src_indexes)
src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
trg_indexes = [vocab['<sos>']]
trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1).to(device)
with torch.no_grad():
  output = model(src_tensor, trg_tensor)
print(output)
output_dim = output.shape[-1]
output = output[1:].reshape(-1, output_dim)

[26, 7, 3, 6, 15, 20, 27, 12, 19, 3, 18, 4, 4, 10, 13, 4, 8]
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.,

In [ ]:
predicted_tokens = torch.argmax(output, dim=1)

predicted_sentence = [itos[tok.item()] for tok in predicted_tokens]
predicted_sentence

[]

In [ ]:
with torch.no_grad():
    for src, trg in test_loader:
        src = src.permute(1, 0).to(device).long()
        trg = trg.permute(1, 0).to(device).long()

        # Turn off teacher forcing
        output = model(src, trg, 0)
        output_dim = output.shape[-1]
        output = output[1:].reshape(-1, output_dim)

        # Convert model's outputs to tokens
        predicted_tokens = torch.argmax(output, dim=1)
        predicted_sentence = ''.join([itos[tok.item()] for tok in predicted_tokens])

        reference = ''.join([itos[tok.item()] for tok in trg[1:].reshape(-1)])
        print(predicted_sentence)
        print(reference)
        break

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!python --version

Python 3.10.12


In [ ]:
!nvidia-smi

Fri Dec 22 01:31:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10                     On  | 00000000:07:00.0 Off |                    0 |
|  0%   30C    P8              15W / 150W |      4MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           196Gi       1.0Gi       194Gi       5.0Mi       1.2Gi       194Gi
Swap:             0B          0B          0B


In [ ]:
 !python -c 'import torch ; print("Is available: ", torch.cuda.is_available()) ; print("Current Device: ", torch.cuda.current_device()) ; print("Pytorch CUDA Compiled version: ", torch._C._cuda_getCompiledVersion()) ; print("Pytorch version: ", torch.version) ; print("pytorch file: ", torch.__file__)'

Is available:  True
Current Device:  0
Pytorch CUDA Compiled version:  12010
Pytorch version:  <module 'torch.version' from '/home/ubuntu/.local/lib/python3.10/site-packages/torch/version.py'>
pytorch file:  /home/ubuntu/.local/lib/python3.10/site-packages/torch/__init__.py


In [ ]:
!pip install fastprogress

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
!pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 121.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 125.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 136.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 21.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 55.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 79.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 55.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 41.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 38.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import csv
import torch
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

import pandas as pd

import torch.nn as nn
import torch.optim as optim

import numpy as np
import h5py

from fastprogress import master_bar, progress_bar

In [ ]:
# Custom char tokenizer
def char_tokenizer(text):
    return list(text)

# Yield tokens from dataset for vocabulary building
def yield_tokens(data):
    for src, trg in data:
        yield char_tokenizer(src)
        yield char_tokenizer(trg)

# Load data from CSV
def load_data_from_csv(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        data = [(row[0], row[1]) for row in csv_reader]
    return data

In [ ]:
train_data_raw = load_data_from_csv('train.csv')
val_data_raw = load_data_from_csv('val.csv')
test_data_raw = load_data_from_csv('test.csv')

In [ ]:
vocab = build_vocab_from_iterator(yield_tokens(train_data_raw), specials=['<sos>', '<eos>', '<unk>'])
vocab.set_default_index(vocab["<unk>"])

print(f"Vocab length: {len(vocab)}")

Vocab length: 4131


In [ ]:
# save vocab
import pickle

with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [ ]:
# Dataset class
class CSVDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        src, trg = self.data[index]
        src = [self.vocab[token] for token in char_tokenizer(src)]
        trg = [self.vocab['<sos>']] + [self.vocab[token] for token in char_tokenizer(trg)]  # prepend <sos>
        return torch.tensor(src), torch.tensor(trg)

In [ ]:
# Padding function
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, padding_value=vocab['<eos>'], batch_first=True)
    trg_batch = pad_sequence(trg_batch, padding_value=vocab['<eos>'], batch_first=True)
    return src_batch, trg_batch

In [ ]:
# DataLoader
BATCH_SIZE = 512

train_dataset = CSVDataset(train_data_raw, vocab)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)

val_dataset = CSVDataset(val_data_raw, vocab)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)  # Usually, no need to shuffle val/test data

test_dataset = CSVDataset(test_data_raw, vocab)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)

In [ ]:
# check sample from dataset
for src, trg in train_dataset:
    print(src, trg)
    break

tensor([ 3,  7, 14, 16,  7,  4, 15, 21, 21, 12,  7, 16, 23,  6, 10, 11,  5, 14]) tensor([ 0, 14,  5, 41, 16,  6,  5,  7,  4, 15])


In [ ]:
# Sample usage:
for src, trg in train_loader:
    # src and trg here are tokenized, batched, and padded sequences
    print(src.shape, trg.shape)
    break

torch.Size([512, 42]) torch.Size([512, 31])


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len, batch_size = trg.shape
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[0, :]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = torch.rand(1) < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from IPython.utils import io

# Assuming src and trg are your tokenized, padded source and target sequences
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)

lr = 0.001

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60, eta_min=1e-5)
criterion = nn.CrossEntropyLoss()
early_stopper = EarlyStopper(patience=10)

epochs=5

mb = master_bar(range(epochs))
mb.names = ['Training loss', 'Validation loss']

x = []
training_losses = []
validation_losses = []

valid_mean_min = np.Inf

# Training loop
for epoch in mb:
  x.append(epoch)
  with io.capture_output() as captured:
      %store x
  total_loss = torch.Tensor([0.0]).to(device)

  # train
  total_train_batches = len(train_loader)
  with io.capture_output() as captured:
      %store total_train_batches
  tbc = 0
  model.train()
  for batch in progress_bar(train_loader, parent=mb):
      tbc = tbc + 1
      with io.capture_output() as captured:
          %store tbc
      src, trg = batch
      src = src.permute(1, 0).to(device).long()
      trg = trg.permute(1, 0).to(device).long()
      optimizer.zero_grad()
      output = model(src, trg)
      output_dim = output.shape[-1]
      output = output[1:].reshape(-1, output_dim)
      trg = trg[1:].reshape(-1)
      loss = criterion(output, trg)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()

  # decay lr
  scheduler.step()
  mean = total_loss / len(train_loader)
  training_losses.append(mean.cpu())
  with io.capture_output() as captured:
      %store training_losses

  # validation
  model.eval()
  total_val_batches = len(val_loader)
  with io.capture_output() as captured:
      %store total_val_batches
  validation_loss = torch.Tensor([0.0]).to(device)
  vbc = 0
  with torch.no_grad():
    for batch in progress_bar(val_loader, parent=mb):
      vbc = vbc + 1
      with io.capture_output() as captured:
          %store vbc
      src, trg = batch
      src = src.permute(1, 0).to(device).long()
      trg = trg.permute(1, 0).to(device).long()
      output = model(src, trg)
      output_dim = output.shape[-1]
      output = output[1:].reshape(-1, output_dim)
      trg = trg[1:].reshape(-1)
      loss = criterion(output, trg)
      validation_loss += loss.item()

  val_mean = validation_loss / len(val_loader)
  validation_losses.append(val_mean.cpu())
  with io.capture_output() as captured:
      %store validation_losses
  mb.update_graph([[x, training_losses], [x, validation_losses]], [0,epochs])
  mb.write(f"\nEpoch {epoch}: Training loss {mean.item():.6f} validation loss {val_mean.item():.6f} with lr {lr:.6f}")

  # save model if validation loss has decreased
  if val_mean.item() <= valid_mean_min:
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
      valid_mean_min,
      val_mean.item()))
      torch.save(model.state_dict(), 'pass_predict.pt')
      valid_mean_min = val_mean.item()

  if early_stopper.early_stop(val_mean.item()):
    break

In [ ]:
print("done")

done


In [ ]:
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
model.load_state_dict(torch.load('pass_predict.pt'))

<All keys matched successfully>

In [ ]:
itos = vocab.get_itos()

In [ ]:
itos[0]

'<sos>'

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

model.eval()
total_bleu = 0

with torch.no_grad():
    for src, trg in test_loader:
        src = src.permute(1, 0).to(device).long()
        trg = trg.permute(1, 0).to(device).long()

        # Turn off teacher forcing
        output = model(src, trg, 0)
        output_dim = output.shape[-1]
        output = output[1:].reshape(-1, output_dim)

        # Convert model's outputs to tokens
        predicted_tokens = torch.argmax(output, dim=1)
        predicted_sentence = [itos[tok.item()] for tok in predicted_tokens]

        reference = [itos[tok.item()] for tok in trg[1:].reshape(-1)]
        total_bleu += sentence_bleu([reference], predicted_sentence)

average_bleu = total_bleu / len(test_loader)
print(f'Average BLEU score: {average_bleu:.3f}')

Average BLEU score: 0.719


In [ ]:
def predict(model, vocab, device, src_sentence, n=1):
    model.eval()

    # Convert the source sentence to tokens
    src_tokens = [tok for tok in src_sentence]
    src_indexes = [vocab[tok] for tok in src_tokens] + [vocab['<eos>']]

    # Convert to tensor and add batch dimension
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # Initialize target sequence with <sos> token
    result = []
    for i in range(n):
        trg_indexes = [vocab['<sos>']]
        predicted_sentence = []
        # Predict next token
        while True:
            trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1).to(device)
            with torch.no_grad():
                output = model(src_tensor, trg_tensor)
                #print(output)
                predicted_token = output.argmax(2)[-1, :].item()
                # Check for end of sentence
                # Append the predicted token to the target sequence
                trg_indexes.append(predicted_token)
                if predicted_token == vocab['<eos>']:
                    break

                # Append the predicted token to the target sequence
                trg_indexes.append(predicted_token)
                # Convert the target sequence to words
                predicted_sentence.append(itos[predicted_token])
        if ''.join(predicted_sentence[1:]) in result:
            continue
        result.append(''.join(predicted_sentence[1:]))

    return result  # Exclude the <sos> token

In [ ]:
predict(model, vocab, device, "pyro.archvile@googlemail.com", 100)

['aere',
 'aels',
 'ailer',
 'aell',
 'ana',
 'aie',
 'aea',
 'ail',
 'anc',
 'aia',
 'aee',
 'ain',
 'aeon',
 'aio',
 'ane',
 'aear',
 'ano',
 'aier',
 'aeer',
 'anar',
 'ance',
 'aic',
 'aer',
 'aera',
 'ael',
 'anan']

In [ ]:
!head -10 /content/drive/MyDrive/Colab/password_predict/data/test.csv

user,pass
bikramojha@yandex.ru,bhp455614
adjoa.mansa@yahoo.com,rbsfdplc
pyro.archvile@googlemail.com,Omega666
red_head76_2000@yahoo.com,0002_67daeh_der
laurentnancy@hotmail.fr,27091978zxc
19ol64@mail.ru,785333
grae123@yahoo.com,enzofan1
gulnora_nabiyeva@yahoo.com,1qaz2wsx3edc
wydik@ya.ru,525252


In [ ]:
model.eval()
src_sentence = "grae123@yahoo.com"
src_tokens = [tok for tok in src_sentence]
src_indexes = [vocab[tok] for tok in src_tokens]
print(src_indexes)
src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
trg_indexes = [vocab['<sos>']]
trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1).to(device)
with torch.no_grad():
  output = model(src_tensor, trg_tensor)
print(output)
output_dim = output.shape[-1]
output = output[1:].reshape(-1, output_dim)

[26, 7, 3, 6, 15, 20, 27, 12, 19, 3, 18, 4, 4, 10, 13, 4, 8]
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.,

In [ ]:
predicted_tokens = torch.argmax(output, dim=1)

predicted_sentence = [itos[tok.item()] for tok in predicted_tokens]
predicted_sentence

[]

In [ ]:
with torch.no_grad():
    for src, trg in test_loader:
        src = src.permute(1, 0).to(device).long()
        trg = trg.permute(1, 0).to(device).long()

        # Turn off teacher forcing
        output = model(src, trg, 0)
        output_dim = output.shape[-1]
        output = output[1:].reshape(-1, output_dim)

        # Convert model's outputs to tokens
        predicted_tokens = torch.argmax(output, dim=1)
        predicted_sentence = ''.join([itos[tok.item()] for tok in predicted_tokens])

        reference = ''.join([itos[tok.item()] for tok in trg[1:].reshape(-1)])
        print(predicted_sentence)
        print(reference)
        break

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!python --version

Python 3.10.12


In [ ]:
!nvidia-smi

Fri Dec 22 01:31:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10                     On  | 00000000:07:00.0 Off |                    0 |
|  0%   30C    P8              15W / 150W |      4MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           196Gi       1.0Gi       194Gi       5.0Mi       1.2Gi       194Gi
Swap:             0B          0B          0B


In [ ]:
 !python -c 'import torch ; print("Is available: ", torch.cuda.is_available()) ; print("Current Device: ", torch.cuda.current_device()) ; print("Pytorch CUDA Compiled version: ", torch._C._cuda_getCompiledVersion()) ; print("Pytorch version: ", torch.version) ; print("pytorch file: ", torch.__file__)'

Is available:  True
Current Device:  0
Pytorch CUDA Compiled version:  12010
Pytorch version:  <module 'torch.version' from '/home/ubuntu/.local/lib/python3.10/site-packages/torch/version.py'>
pytorch file:  /home/ubuntu/.local/lib/python3.10/site-packages/torch/__init__.py


In [ ]:
!pip install fastprogress

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
!pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 121.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 125.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 136.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 21.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 55.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 79.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 55.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 41.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 38.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import csv
import torch
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

import pandas as pd

import torch.nn as nn
import torch.optim as optim

import numpy as np
import h5py

from fastprogress import master_bar, progress_bar

In [ ]:
# Custom char tokenizer
def char_tokenizer(text):
    return list(text)

# Yield tokens from dataset for vocabulary building
def yield_tokens(data):
    for src, trg in data:
        yield char_tokenizer(src)
        yield char_tokenizer(trg)

# Load data from CSV
def load_data_from_csv(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        data = [(row[0], row[1]) for row in csv_reader]
    return data

In [ ]:
train_data_raw = load_data_from_csv('train.csv')
val_data_raw = load_data_from_csv('val.csv')
test_data_raw = load_data_from_csv('test.csv')

In [ ]:
vocab = build_vocab_from_iterator(yield_tokens(train_data_raw), specials=['<sos>', '<eos>', '<unk>'])
vocab.set_default_index(vocab["<unk>"])

print(f"Vocab length: {len(vocab)}")

Vocab length: 4131


In [ ]:
# save vocab
import pickle

with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [ ]:
# Dataset class
class CSVDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        src, trg = self.data[index]
        src = [self.vocab[token] for token in char_tokenizer(src)]
        trg = [self.vocab['<sos>']] + [self.vocab[token] for token in char_tokenizer(trg)]  # prepend <sos>
        return torch.tensor(src), torch.tensor(trg)

In [ ]:
# Padding function
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, padding_value=vocab['<eos>'], batch_first=True)
    trg_batch = pad_sequence(trg_batch, padding_value=vocab['<eos>'], batch_first=True)
    return src_batch, trg_batch

In [ ]:
# DataLoader
BATCH_SIZE = 512

train_dataset = CSVDataset(train_data_raw, vocab)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)

val_dataset = CSVDataset(val_data_raw, vocab)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)  # Usually, no need to shuffle val/test data

test_dataset = CSVDataset(test_data_raw, vocab)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)

In [ ]:
# check sample from dataset
for src, trg in train_dataset:
    print(src, trg)
    break

tensor([ 3,  7, 14, 16,  7,  4, 15, 21, 21, 12,  7, 16, 23,  6, 10, 11,  5, 14]) tensor([ 0, 14,  5, 41, 16,  6,  5,  7,  4, 15])


In [ ]:
# Sample usage:
for src, trg in train_loader:
    # src and trg here are tokenized, batched, and padded sequences
    print(src.shape, trg.shape)
    break

torch.Size([512, 42]) torch.Size([512, 31])


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len, batch_size = trg.shape
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[0, :]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = torch.rand(1) < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from IPython.utils import io

# Assuming src and trg are your tokenized, padded source and target sequences
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)

lr = 0.001

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60, eta_min=1e-5)
criterion = nn.CrossEntropyLoss()
early_stopper = EarlyStopper(patience=10)

epochs=5

mb = master_bar(range(epochs))
mb.names = ['Training loss', 'Validation loss']

x = []
training_losses = []
validation_losses = []

valid_mean_min = np.Inf

# Training loop
for epoch in mb:
  x.append(epoch)
  with io.capture_output() as captured:
      %store x
  total_loss = torch.Tensor([0.0]).to(device)

  # train
  total_train_batches = len(train_loader)
  with io.capture_output() as captured:
      %store total_train_batches
  tbc = 0
  model.train()
  for batch in progress_bar(train_loader, parent=mb):
      tbc = tbc + 1
      with io.capture_output() as captured:
          %store tbc
      src, trg = batch
      src = src.permute(1, 0).to(device).long()
      trg = trg.permute(1, 0).to(device).long()
      optimizer.zero_grad()
      output = model(src, trg)
      output_dim = output.shape[-1]
      output = output[1:].reshape(-1, output_dim)
      trg = trg[1:].reshape(-1)
      loss = criterion(output, trg)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()

  # decay lr
  scheduler.step()
  mean = total_loss / len(train_loader)
  training_losses.append(mean.cpu())
  with io.capture_output() as captured:
      %store training_losses

  # validation
  model.eval()
  total_val_batches = len(val_loader)
  with io.capture_output() as captured:
      %store total_val_batches
  validation_loss = torch.Tensor([0.0]).to(device)
  vbc = 0
  with torch.no_grad():
    for batch in progress_bar(val_loader, parent=mb):
      vbc = vbc + 1
      with io.capture_output() as captured:
          %store vbc
      src, trg = batch
      src = src.permute(1, 0).to(device).long()
      trg = trg.permute(1, 0).to(device).long()
      output = model(src, trg)
      output_dim = output.shape[-1]
      output = output[1:].reshape(-1, output_dim)
      trg = trg[1:].reshape(-1)
      loss = criterion(output, trg)
      validation_loss += loss.item()

  val_mean = validation_loss / len(val_loader)
  validation_losses.append(val_mean.cpu())
  with io.capture_output() as captured:
      %store validation_losses
  mb.update_graph([[x, training_losses], [x, validation_losses]], [0,epochs])
  mb.write(f"\nEpoch {epoch}: Training loss {mean.item():.6f} validation loss {val_mean.item():.6f} with lr {lr:.6f}")

  # save model if validation loss has decreased
  if val_mean.item() <= valid_mean_min:
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
      valid_mean_min,
      val_mean.item()))
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab/password_predict/pass_predict.pt')
      valid_mean_min = val_mean.item()

  if early_stopper.early_stop(val_mean.item()):
    break

In [ ]:
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab/password_predict/pass_predict.pt'))

<All keys matched successfully>

In [ ]:
itos = vocab.get_itos()

In [ ]:
itos[0]

'<sos>'

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

model.eval()
total_bleu = 0

with torch.no_grad():
    for src, trg in test_loader:
        src = src.permute(1, 0).to(device).long()
        trg = trg.permute(1, 0).to(device).long()

        # Turn off teacher forcing
        output = model(src, trg, 0)
        output_dim = output.shape[-1]
        output = output[1:].reshape(-1, output_dim)

        # Convert model's outputs to tokens
        predicted_tokens = torch.argmax(output, dim=1)
        predicted_sentence = [itos[tok.item()] for tok in predicted_tokens]

        reference = [itos[tok.item()] for tok in trg[1:].reshape(-1)]
        total_bleu += sentence_bleu([reference], predicted_sentence)

average_bleu = total_bleu / len(test_loader)
print(f'Average BLEU score: {average_bleu:.3f}')

Average BLEU score: 0.719


In [ ]:
def predict(model, vocab, device, src_sentence, n=1):
    model.eval()

    # Convert the source sentence to tokens
    src_tokens = [tok for tok in src_sentence]
    src_indexes = [vocab[tok] for tok in src_tokens] + [vocab['<eos>']]

    # Convert to tensor and add batch dimension
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # Initialize target sequence with <sos> token
    result = []
    for i in range(n):
        trg_indexes = [vocab['<sos>']]
        predicted_sentence = []
        # Predict next token
        while True:
            trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1).to(device)
            with torch.no_grad():
                output = model(src_tensor, trg_tensor)
                #print(output)
                predicted_token = output.argmax(2)[-1, :].item()
                # Check for end of sentence
                # Append the predicted token to the target sequence
                trg_indexes.append(predicted_token)
                if predicted_token == vocab['<eos>']:
                    break

                # Append the predicted token to the target sequence
                trg_indexes.append(predicted_token)
                # Convert the target sequence to words
                predicted_sentence.append(itos[predicted_token])
        if ''.join(predicted_sentence[1:]) in result:
            continue
        result.append(''.join(predicted_sentence[1:]))

    return result  # Exclude the <sos> token

In [ ]:
predict(model, vocab, device, "pyro.archvile@googlemail.com", 100)

['aere',
 'aels',
 'ailer',
 'aell',
 'ana',
 'aie',
 'aea',
 'ail',
 'anc',
 'aia',
 'aee',
 'ain',
 'aeon',
 'aio',
 'ane',
 'aear',
 'ano',
 'aier',
 'aeer',
 'anar',
 'ance',
 'aic',
 'aer',
 'aera',
 'ael',
 'anan']

In [ ]:
!head -10 /content/drive/MyDrive/Colab/password_predict/data/test.csv

user,pass
bikramojha@yandex.ru,bhp455614
adjoa.mansa@yahoo.com,rbsfdplc
pyro.archvile@googlemail.com,Omega666
red_head76_2000@yahoo.com,0002_67daeh_der
laurentnancy@hotmail.fr,27091978zxc
19ol64@mail.ru,785333
grae123@yahoo.com,enzofan1
gulnora_nabiyeva@yahoo.com,1qaz2wsx3edc
wydik@ya.ru,525252


In [ ]:
model.eval()
src_sentence = "grae123@yahoo.com"
src_tokens = [tok for tok in src_sentence]
src_indexes = [vocab[tok] for tok in src_tokens]
print(src_indexes)
src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
trg_indexes = [vocab['<sos>']]
trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(1).to(device)
with torch.no_grad():
  output = model(src_tensor, trg_tensor)
print(output)
output_dim = output.shape[-1]
output = output[1:].reshape(-1, output_dim)

[26, 7, 3, 6, 15, 20, 27, 12, 19, 3, 18, 4, 4, 10, 13, 4, 8]
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.,

In [ ]:
predicted_tokens = torch.argmax(output, dim=1)

predicted_sentence = [itos[tok.item()] for tok in predicted_tokens]
predicted_sentence

[]

In [ ]:
with torch.no_grad():
    for src, trg in test_loader:
        src = src.permute(1, 0).to(device).long()
        trg = trg.permute(1, 0).to(device).long()

        # Turn off teacher forcing
        output = model(src, trg, 0)
        output_dim = output.shape[-1]
        output = output[1:].reshape(-1, output_dim)

        # Convert model's outputs to tokens
        predicted_tokens = torch.argmax(output, dim=1)
        predicted_sentence = ''.join([itos[tok.item()] for tok in predicted_tokens])

        reference = ''.join([itos[tok.item()] for tok in trg[1:].reshape(-1)])
        print(predicted_sentence)
        print(reference)
        break